# Part II - (The determinant of interest rate and loan amount from Prosper Loan Data Analysis)
## by (Mutholib Yusira)

## Investigation Overview


> Prosper is a money lending platform which helps in reducing the distance between the borrower and lender(peer-to-peer lending process). The objective of this work is to look at the main determinants of loans and borrowers interest rate that could be used to predict their evolution. The main focus was on several borrowers information, such as income, occupation, interest rate ....


## Dataset Overview

> The Prosper loan dataset comprises of 113937 loan entries with 81 attributes on each loan, including loan amount, borrower rate (or interest rate), current loan status, borrower income, and many others, from the year 2009-2014. There are two main categories:

> * Borrower information: Basic attributes of the borrowers such as annual income, condition of employment, interest rate, loan status, etc.
> * Loan performance information: Metrics evaluating the risk associated with the loans such as Prosper score and bank card utilization, etc.

> There were some elements that need to be fixed, in order to create interesting and trustworthy analyses and visualizations.

In [ ]:
# import all packages and set plots to be embedded inline
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [ ]:
# load in the dataset into a pandas dataframe
loan_data = pd.read_csv('prosperLoanData.csv')

In [ ]:
#Selecting relevant columns for this analysis
r_columns = ['ListingNumber', 'ListingCreationDate', 'Term', 'LoanStatus', 'BorrowerAPR', 'BorrowerRate', \
            'ProsperRating (Alpha)', 'ProsperScore', 'ListingCategory (numeric)', 'BorrowerState', 'Occupation', \
            'EmploymentStatus', 'IsBorrowerHomeowner', 'CreditScoreRangeLower', 'CreditScoreRangeUpper', \
            'CurrentCreditLines', 'OpenRevolvingAccounts', 'DebtToIncomeRatio', 'IncomeRange', 'IncomeVerifiable', \
            'StatedMonthlyIncome', 'LoanOriginalAmount', 'LoanOriginationDate',]

In [ ]:
#Creating a sub dataset of the 'loan_data', 'df'
df = loan_data[r_columns]

In [ ]:
#dropping rows with null values
df = df.dropna()

In [ ]:
#renaming specific columns that have spaces in them
df.rename(columns={'ProsperRating (Alpha)':'ProsperRating', 'ListingCategory (numeric)':'ListingCategory'}, inplace=True)

In [ ]:
#matching the ListingCategory values to their meanings as provided in the context of the dataset
list_dict = {0 : 'Not Available', 1 : 'Debt Consolidation', 2 : 'Home Improvement', 3: 'Business', 
             4 : 'Personal Loan', 5 : 'Student Use', 6 : 'Auto', 7 : 'Other', 8 : 'Baby&Adoption', 
             9 : 'Boat', 10 : 'Cosmetic Procedure', 11 : 'Engagement Ring', 12 : 'Green Loans',
             13 : 'Household Expenses', 14 : 'Large Purchases', 15 : 'Medical/Dental', 16 : 'Motorcycle',
             17 : 'RV', 18 : 'Taxes', 19 : 'Vacation', 20 : 'Wedding Loans'}

df['ListingCategory'] = df['ListingCategory'].map(list_dict)

In [ ]:
#converting columns to the write datatypes
df['ListingNumber'] = df['ListingNumber'].astype(str)
df['ListingCreationDate'] = pd.to_datetime(df['ListingCreationDate'])
df['LoanStatus'] = df['LoanStatus'].astype('category')
df['ProsperScore'] = df['ProsperScore'].astype(int)
df['CreditScoreRangeLower'] = df['CreditScoreRangeLower'].astype(int)
df['CreditScoreRangeUpper'] = df['CreditScoreRangeUpper'].astype(int)
df['CurrentCreditLines'] = df['CurrentCreditLines'].astype(int)
df['IncomeRange'] = df['IncomeRange'].astype('category')
df['LoanOriginationDate'] = pd.to_datetime(df['LoanOriginationDate'])

In [ ]:
#converting 'ProsperRating' column to ordinal category type
rating = ['AA', 'A', 'B', 'C', 'D', 'E','HR']
ordered_var = pd.api.types.CategoricalDtype(ordered = True, categories = rating)
df['ProsperRating'] = df['ProsperRating'].astype(ordered_var)

In [ ]:
#Extracting date, month and year information from 'ListingCreationDate' column
df['Date'] = df['ListingCreationDate'].dt.date
df['Month']  = df['ListingCreationDate'].dt.month
df['Year'] = df['ListingCreationDate'].dt.year

In [ ]:
#dropping 'ListingCreationDate' column
df.drop(['ListingCreationDate'], axis=1, inplace=True)

In [ ]:
#Converting date column dtype
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
#changing the numerical values of the 'Month' column to month names
month_dict = {1 : 'January', 2 : 'February', 3: 'March', 
             4 : 'April', 5 : 'May', 6 : 'June', 7 : 'July', 8 : 'August', 
             9 : 'September', 10 : 'October', 11 : 'November', 12 : 'December'}

df['Month'] = df['Month'].map(month_dict)

In [ ]:
#creating a new column 'CreditScoreAverage' by getting an average of the credit scores provided
df['CreditScoreAverage'] = (df['CreditScoreRangeUpper'] + df['CreditScoreRangeLower'])/2

In [ ]:
#Feature engineering of a new column 'Term_yrs' from 'Term' to convert the term in months to years
df['Term_yrs'] = df['Term']/12
df['Term_yrs'] = df['Term_yrs'].astype(int)

In [ ]:
# define a classification function
def classify(income):
    if income["StatedMonthlyIncome"]<2500:
        return 'Low'
    if income["StatedMonthlyIncome"]<8000:
        return 'Medium'
    else:
        return 'High'
    
df['IncomeGroup'] = df.apply(classify, axis=1)

# convert string to ordinal category type
income_order = ['Low', 'Medium', 'High']
income_ordered_variable = pd.api.types.CategoricalDtype(ordered = True, categories = income_order)
df['IncomeGroup'] = df['IncomeGroup'].astype(income_ordered_variable)

> Note that the above cells have been set as "Skip"-type slides. That means
that when the notebook is rendered as http slides, those cells won't show up.

## Loan Original Amount Distribution
There are 3 peaks of the Loan Original Amount, at 4k, 10k and 15k. These 3 loan amounts are the most common loan amounts borrowed with 4k being the highest

In [ ]:
color_base = sb.color_palette()[2]
log_binsize = 0.025
bins_log = 10 ** np.arange(3, np.log10(df['LoanOriginalAmount'].max())+0.025, 0.025)
plt.figure(figsize=(14,8))
plt.hist(data = df, x = 'LoanOriginalAmount', bins = bins_log, color=color_base)
plt.xscale('log')
x_ticks = [1000, 2000, 3000, 4000, 5000, 10000, 15000,20000, 30000]
x_ticks_names = ['1k','2k', '3k', '4k', '5k', '10k','15k', '20k','30k']
plt.xticks(x_ticks, x_ticks_names)
plt.xlim((1000,35000))
plt.xlabel('Loan Original Amount ($)')
plt.title('Distribution of Loan Original Amount ($)');

## Loan Status Distribution

67.1% of the loan payments are current, 23.2% are completed. That puts approximately 80% of the loans in the dataset in good profiles. Only 2.4% of the loans that were grouped together are past due and 1.2% of them defaulted. 

In [ ]:
# Collapsing Past Due loans 
df['LoanStatus'] = df['LoanStatus'].apply(lambda x: x if 'Past Due' not in x else 'Past Due')

# Convert LoanStatus to a categorical variable
status_dict = ['Defaulted','Chargedoff', 'Past Due', 'Current', 'FinalPaymentInProgress', 'Completed']

loan_status_order = pd.api.types.CategoricalDtype(ordered = True, categories = status_dict)

df['LoanStatus'] = df['LoanStatus'].astype(loan_status_order)

In [ ]:
# Printing the proportion above the bars 
n_loans=df['LoanStatus'].value_counts().sum()
status_count=df['LoanStatus'].value_counts()
status_order=status_count.index

plt.figure(figsize=(14,8))
sb.countplot(data=df, x='LoanStatus', color=color_base, order=status_order);

plt.xlabel('Count')
plt.ylabel('Loan Status')
plt.title('Loan Status Distribution')

# getting the current tick locations and labels
locs, labels = plt.xticks(rotation=0) 

# looping through each pair of locations and labels
for loc, label in zip(locs, labels):
    # getting the text property for the label to get the correct count
    count = status_count[label.get_text()]
    pct_string = '{:0.1f}%'.format(100*count/n_loans)

    # printing the annotation just below the top of the bar
    plt.text(loc, count+2, pct_string, ha = 'center', color = 'black')

## Relationship Between Borrower Rate and Employment Status

Borrowers with full time jobs and with a stable source of income (Employed, Full-time, SElf-employed), had lower interest loans that others. There is only one unemployed borrower in the dataset, and that person has a very high interest rate compared to most of the borrowers.

In [ ]:
plt.figure(figsize=(14,8))

sb.boxplot(x='EmploymentStatus', y='BorrowerRate', data=df, color=color_base)
plt.xticks(rotation=15)

plt.xlabel('Employment Status')
plt.ylabel('Borrower Rate(%)')
plt.title('Relationship Between Borrower Rate and Employment Status');

## Relationship Between Borrower Rate and Home Owner Status

Borrowers who are homeowners and possibly use those homes as collateral have a lower interest rate than borrowers who do not have homes.

In [ ]:
plt.figure(figsize=(14,8))

sb.boxplot(x='IsBorrowerHomeowner', y='BorrowerRate', data=df, color=color_base)
plt.xlabel('Home Owner Status')
plt.ylabel('Borrower Rate(%)')
plt.title('Relationship Between Borrower Rate and Home Owner Status');

## Relationship Between Borrower Rate and Income Group

There is a negative correlation between the income groups of the borrowers and the interest rate on the loans they take out. The high income group have the lowest loan interest rate, and the low income group have the highest interest rate.

In [ ]:
plt.figure(figsize=(14,8))

sb.boxplot(x='IncomeGroup', y='BorrowerRate', data=df, color=color_base)
plt.xlabel('Income Group')
plt.ylabel('Borrower Rate(%)')
plt.title('Relationship Between Borrower Rate and Income Group');

## Relationship Between Borrower Rate and Prosper Score, Rating

Either the Prosper Score or Prosper Rating is a really good determinant of the loan interest rate. The higher the Prosper Score, the lower the interest rate on the loans. Higher Prosper Ratings are also associated with lower loan interest rates.

In [ ]:
#define violin function
def violin(var, color):
    fig, ax = plt.subplots(ncols = 2, figsize = [14, 8])
    sb.violinplot(data = df, y = var, x = 'ProsperScore', ax = ax[0],
                  color = color)
    ax[0].set_xlabel("Prosper Score")
    ax[0].set_ylabel(var.capitalize())
    sb.violinplot(data = df, y = var, x = 'ProsperRating', ax = ax[1],
               color = color)

    ax[1].set_xlabel("Prosper Rating")
    ax[1].set_ylabel("")
    fig.suptitle('Relationship between Borrower Rate and Prosper Score, Prosper Rating')
    plt.show()
    
violin('BorrowerRate', sb.color_palette()[2])

## Relationship Between Loan Amount And Income Group, Employment Status

Borrowers who are steadily employed(Employed, Full-time, Self-employed) and who are in high income groups have access to higher amounts for loans

In [ ]:
#define violin function
def boxgrid(var, color):
    fig, ax = plt.subplots(ncols = 2, figsize = [14,8])
    sb.boxplot(data = df, y = var, x = 'IncomeGroup', ax = ax[0],
                  color = color)
    ax[0].set_xlabel("Income Group",fontsize=20)
    ax[0].set_ylabel(var.capitalize(), fontsize=20)
    sb.boxplot(data = df, y = var, x = 'EmploymentStatus', ax = ax[1],
               color = color)
    ax[1].set_xlabel("Employment Status",fontsize=20)
    ax[1].set_ylabel("")
    plt.xticks(rotation=45)
    fig.suptitle('Relationship between Loan Amount and Income Group, Employment status', fontsize=30)
    plt.show()
    
boxgrid('LoanOriginalAmount', sb.color_palette()[2])

## Relationship Between Loan Amount And Home Owner Status

Borrowers who are home owners have access to higher amounts of loans, that those who do not own homes

In [ ]:
plt.figure(figsize=(14,8))

sb.boxplot(x='IsBorrowerHomeowner', y='LoanOriginalAmount', data=df, color=color_base)
plt.xlabel('Home Owner Status')
plt.ylabel('Loan Original Amount ($)')
plt.title('Relationship Between Loan Amount And Home Owner Status');

## Borrower Rate and Average Credit Score by Loan Payment Status

For people who are delinquent (defaulted,past due, charged-off payments), banks applies more restricted credit conditions (higher interest rates). Also these people have lower credit scores than people with good status.

In [ ]:
#Create a new column which return 1 for good status of payment('Current','Completed','FinalPaymentInProgress') 
# and 0 if there is an issue (defaulted,past due, charged-off payments)

status_good=['Current','Completed','FinalPaymentInProgress']
def cat_Loanstatus(row):
    for i in status_good:
        if i in row:
            return 1
    return 0
df["loanstatus_cat"] = df['LoanStatus'].apply(lambda x: cat_Loanstatus(x))

In [ ]:
status = df.groupby('loanstatus_cat').agg({'CreditScoreAverage':'mean','BorrowerRate':'mean'})
status = status.reset_index()
status["loanstatus_cat"].replace({0: "Delinquent", 1: "Good"}, inplace=True)
status = status.sort_values(['CreditScoreAverage'],ascending=False).head(10)
status

In [ ]:
# Create a figure
fig = plt.figure(figsize=(14,8))
# Create matplotlib axes
ax = fig.add_subplot(111)
# Create another axes 
ax2 = ax.twinx()

width = 0.20
CreditScore_average = status.plot(x='loanstatus_cat', y='CreditScoreAverage', kind='bar', color = 'green',
                   width=width,ax=ax, position = 0, label = 'Average CreditScore')
BorrowerRate = status.plot(x='loanstatus_cat', y='BorrowerRate', kind='bar', color = 'orange',
                  width=width,ax=ax2, position = 1,  label = 'Borrower Rate')

ax.set_ylabel('Credit Score Average')
CreditScore_average.legend(loc='upper right')
ax2.set_ylabel('Borrower Rate')
ax2.legend(loc='upper left')
ax.set_xlabel('Loan Status categories')
plt.title('Borrower Rate and Average Credit Score by Delinquency Status');

## Borrower Rate by Prosper Rating and Loan Terms

The lower the loan term, the lower the interest rate on the loan. As observed earlier, the better the Prosper Rating, the lower the interest rate on the loans. It should also be noted that HR, the least Prosper Rating has only 3 years loan terms.

In [ ]:
fig = plt.figure(figsize = [14,8])
ax = sb.pointplot(data = df, x = 'ProsperRating', y = 'BorrowerRate', hue = 'Term_yrs',
           palette = 'Greens', linestyles = '',dodge = 0.4, ci='sd')
plt.xlabel('Prosper Rating')
plt.ylabel('Borrower Rate(%)')
plt.title('Borrower Rate vs. Prosper Rating by Loan Term in Years');

## Loan Amount by Home Owner Status and Income Group

As shown in the previous slides, having a collateral and a higher wage help getting higher loan amount. It is clearly visible that being a home owner is a very important element to get a higher loan amount.

In [ ]:
plt.figure(figsize = [14,8])
sb.boxplot(data = df, x = 'IncomeGroup', y = 'LoanOriginalAmount', hue = 'IsBorrowerHomeowner', color = color_base)
plt.xlabel('Home Owner Status')
plt.ylabel('Loan Original Amount ($)')
plt.title('Loan Amount vs. Income Group by Home Owner Status');

# Thank You For Your Attention

In [ ]:
!jupyter nbconvert Part_II_slide_deck_template.ipynb --to slides --post serve  --no-input --no-prompt